# **Scraping**
## imports

In [ ]:
from datetime import datetime
from bs4 import BeautifulSoup
import requests
import pandas as pd

## Extraccion de datos
- equipos -> [[equipos_locales], [equipos_visitantes]]
- cuotas -> [[cuota_local, cuota_empate, cuota_visitante]] 
- fecha_hora -> [fecha, hora]
- dobles oportunidades -> [[Lo/Em, Lo/Vi, Vi/Em]]
- ambos_anotan -> [[si, no]]
- goles -> [['RES', "E", "AM", "gol_lo", "X", "gol_vis"]]

In [ ]:
class Scraping:
    def __init__(self, url) -> None:
        print(f"analizando >> {url} <<")
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'lxml')
        self.container = soup.find('div', class_='fragment expander coupon-for-type')

        duplicade_links = [elem.get("href") for elem in self.container.find_all('a', title="Número de mercados")]
        self.links_per_match = [] 

        for item in duplicade_links:
            if item not in self.links_per_match:
                self.links_per_match.append(item)

        #variable para la liga
        self.liga = url.split('/')[-1]

    def equipos(self):
        local_vist = []
        equipos = [elem.text for elem in self.container.find_all('span', class_='seln-name')]
        locales = equipos[::2]
        visitantes = equipos[1::2]
        for i in range(len(locales)):
            match = [locales[i], visitantes[i]]
            local_vist.append(match)
        return local_vist
    
    def cuotas(self):
        puntos = [elem.text for elem in self.container.find_all('span', class_='price dec')]
        cuotas = [puntos[i:i+3] for i in range(0, len(puntos), 3)]
        for i in range(len(cuotas)):
            for j in range(len(cuotas[i])):
                val = cuotas[i][j]
                val = val.replace('.',',')
                cuotas[i][j] = val
        return cuotas

    def hora_fecha(self):
        meses = {'Ene': '01', 'Feb': '02', 'Mar': '03', 'Abr': '04', 'May': '05', 'Jun': '06',
             'Jul': '07', 'Ago': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dic': '12'}
        horarios = []
        eventos = self.container.find_all('div', {'class': 'ev'})
        for evento in eventos:
            hora_element = evento.find('span', {'class': 'time'})
            fecha_element = evento.find('span', {'class': 'date'})

            if hora_element and fecha_element:
                hora = hora_element.text
                fecha = fecha_element.text
                fecha = fecha + ' 2023'
                fecha = fecha.replace(fecha.split()[1], meses[fecha.split()[1]])
                fecha_hora = fecha + ' ' + hora
                fecha_hora = datetime.strptime(fecha_hora, '%d %m %Y %H:%M')
                fecha_hora = datetime.strftime(fecha_hora, '%Y-%m-%d %H:%M')
                horarios.append(str(fecha_hora).split(' '))
        return horarios
    
    def dobles_oportunidades(self):
        dobles_oportunidades = []
        for link in self.links_per_match:
            try:
                new_url = "https://apuestas.wplay.co" + link
                new_page = requests.get(new_url)
                new_soup = BeautifulSoup(new_page.content, 'lxml')
                new_container = new_soup.find('ul', class_='default mkt_content limited mkt-sort-DBLC ev-sort-MT')
                group = [elem.text for elem in new_container.find_all('span', class_="price dec")]
                #remplazando . por ,
                for i in range(len(group)):
                    val = group[i].replace('.',',')
                    group[i] = val
                dobles_oportunidades.append(group)
            except AttributeError:
                print(f".  el link: -- {new_url} -- no contiene dobles_oportunidades")
                dobles_oportunidades.append(["N/A","N/A", "N/A"])
        return dobles_oportunidades
    
    def ambos_anotan(self):
        si_no = []
        for link in self.links_per_match:
            try:
                new_url = "https://apuestas.wplay.co" + link
                new_page = requests.get(new_url)
                new_soup = BeautifulSoup(new_page.content, 'lxml')
                new_container = new_soup.find('table', class_='horizontal mkt_content mkt-sort-BTSC')
                group = [elem.text for elem in new_container.find_all('span', class_="price dec")]
                for i in range(len(group)):
                    val = group[i].replace('.',',')
                    group[i] = val
                si_no.append(group)
            except AttributeError:
                print(f".  el link: -- {new_url} -- no contiene ambos_anotan")
                si_no.append(["N/A","N/A"])
        return si_no
    
    def goles(self):
        resultado = []
        try:
            links = [elem.get("href") for elem in self.container.find_all('a', class_="stats")]
            for link in links:
                page_link = requests.get(link)
                soup_link = BeautifulSoup(page_link.content, 'lxml')
                container_link = soup_link.find('div', class_='sub-container head-to-head')
                puntuacion = container_link.find('div', class_="panel-body padding-medium collapse in")
                goles = puntuacion.find("strong", class_="size-xxxl")
                
                if goles == None:
                    model = ['RES', "E", "AM", "N/A", "X", "N/A"]
                    resultado.append(model)
                else:
                    goles = goles.text.split()
                    uno = int(goles[0])
                    dos = int(goles[2])
                    RES = lambda uno, dos: "EM" if uno == dos else ("GL" if uno > dos else "GV")
                    E = lambda uno, dos: "SI" if uno == dos else "NO"
                    AM = lambda uno, dos : "NO" if uno == 0 and dos == 0 else "SI"
                    resultado.append([RES(uno,dos), E(uno,dos), AM(uno,dos), uno, "X", dos])
            return resultado
        
        except AttributeError as e:
            print("     no contiene partidos en VIVO o pasados")
            print(e)



## creacion del excel

In [ ]:
class Create_Excel():
    def __init__(self, urls) -> None:
        ligas_totales = []
        for url in urls:
            try:
                obj = Scraping(url)
                equipos = obj.equipos()
                cuotas = obj.cuotas()
                hora_fecha = obj.hora_fecha()
                dobles_oportunidades = obj.dobles_oportunidades()
                si_no = obj.ambos_anotan()
                goles = obj.goles()
                self.liga = obj.liga
                
                total_data = zip(equipos, cuotas, hora_fecha,
                                dobles_oportunidades, si_no, goles)
                ligas_totales.extend(self.data_per_liga(total_data))
            except AttributeError:
                print("!NO tiene partidos¡")

            COLUMNS = ['liga', 'local', 'visitante', 'cuo_local', 'cuo_emp', 'cuo_vist', 'fecha', 'hora',
                    'local_emp', 'local_vist', 'vist_emp', 'si_anotan', 'no_anotan',
                    'RES', "E", "AM", "1", "X", "2"]
            
            self.Excel(ligas_totales, COLUMNS)


    def data_per_liga(self, total_data):
        partidos = []
        for i in total_data:
            local, vist = i[0]
            cuo_local, cuo_emp, cuo_vist = i[1]
            fecha, hora = i[2]
            local_emp, local_vist, vist_emp = i[3]
            si_anotan, no_anotan = i[4]
            RES, E, AM, uno, X , dos = i[5]
            datos_part = (self.liga, local, vist, cuo_local, cuo_emp, cuo_vist, fecha, hora,
                          local_emp, local_vist, vist_emp, si_anotan, no_anotan,
                          RES, E, AM, uno, X , dos)
            partidos.append(datos_part)
        return partidos
    
    def Excel(self, data, columns):
        df =pd.DataFrame(data, columns=columns)

        nuevos_nombre = {
            'vist':'visitante',
            'cuo_local':'cuota_local',
            'cuo_emp':'cuota_empate',
            'cuo_vist':'cuota_visitante',
            'local_emp':'local/empate',
            'local_vist':'local/visitante',
            'vist_emp':'visitante/empate'
        }
        df = df.rename(columns=nuevos_nombre)

        columnas_ordenadas = ['liga', 'fecha', 'hora','RES', "E", "AM", "1", "X", "2",
                              'local','cuota_local', 'vs','cuota_empate',
                              'visitante','cuota_visitante','si_anotan', 'no_anotan',
                              'local/empate', 'local/visitante', 'visitante/empate' ]
        df = df.reindex(columns=columnas_ordenadas)

        writer = pd.ExcelWriter('cuotas.xlsx', engine='xlsxwriter')
        df.to_excel(writer, sheet_name='Cuotas_1', index=False)
        writer.close()

## RESULTADOS

In [ ]:
#---ingrese los links en las listas----
total_urls = [
        'https://apuestas.wplay.co/es/PremierLeague',
        'https://apuestas.wplay.co/es/t/19160/La-Liga',
        'https://apuestas.wplay.co/es/Ligue1',
        "https://apuestas.wplay.co/es/ItaliaSerieA",
        'https://apuestas.wplay.co/es/Bundesliga',
        #2sep'https://apuestas.wplay.co/es/t/19156/Inglaterra-Championship',
        'https://apuestas.wplay.co/es/t/19344/Bundesliga-2',
        'https://apuestas.wplay.co/es/PrimeraAColombia',
        #2sep'https://apuestas.wplay.co/es/BrasilserieA',
        'https://apuestas.wplay.co/es/t/19296/Argentina-Copa-de-la-Liga-Profesional',
        'https://apuestas.wplay.co/es/t/19398/Uruguay-Primera-Divisi%C3%B3n',
        'https://apuestas.wplay.co/es/t/19359/Paraguay-Primera-Divisi%C3%B3n',
        #2sep'https://apuestas.wplay.co/es/t/19342/Argentina-Primera-Nacional', 
        'https://apuestas.wplay.co/es/t/19211/Portugal-Primeira-Liga',
        'https://apuestas.wplay.co/es/t/19372/B%C3%A9lgica-1ra-Divisi%C3%B3n-A',
        'https://apuestas.wplay.co/es/t/19358/Holanda-Eredivisie',       
        'https://apuestas.wplay.co/es/MLS',         
        'https://apuestas.wplay.co/es/t/29812/Colombia-Primera-B',
        'https://apuestas.wplay.co/es/t/19373/Ecuador-Primera-A',
        #2sep'https://apuestas.wplay.co/es/t/19340/Per%C3%BA-Primera-Divisi%C3%B3n',
        'https://apuestas.wplay.co/es/t/48352/Segunda',
        'https://apuestas.wplay.co/es/t/19303/Chile-Primera-Divisi%C3%B3n',
        'https://apuestas.wplay.co/es/LigaMX',]

#--ingrese en los parentesis el nombre de la lista---
obj = Create_Excel(total_urls)    
